# How to deploy ADK Apps to Agent Engine

[Documentation link 🔗](https://google.github.io/adk-docs/deploy/agent-engine/)

## 1. Set the local environment variables
These will be used in the Agent Engine deployment

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../trends_and_insights_agent/.env")  # take environment variables

True

## 2. Set up the `AdkApp` Convenience wrapper for ADK Agents -> Agent Engine

Note the injection of the environment variables.

In [2]:
from vertexai.preview.reasoning_engines import AdkApp

from trends_and_insights_agent import agent

env_vars = {}

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["BUCKET"] = os.getenv("BUCKET")
env_vars["GOOGLE_CLOUD_PROJECT_NUMBER"] = os.getenv("GOOGLE_CLOUD_PROJECT_NUMBER")
env_vars["YT_SECRET_MNGR_NAME"] = os.getenv("YT_SECRET_MNGR_NAME")
env_vars["SESSION_STATE_JSON_PATH"] = os.getenv("SESSION_STATE_JSON_PATH")

my_agent = AdkApp(agent=agent.root_agent, enable_tracing=True, env_vars=env_vars)

INFO:root:

`SESSION_STATE_JSON_PATH`: example_state_prs.json


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
/Users/jwortz/zghost/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/jwortz/zghost/trends_and_insights_agent/common_agents/trend_assistant/tools.py:134: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return max_date.iloc[0][0].strftime("%m/%d/%Y")
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


# 3. Test the Agent Engine locally

In [3]:
session = my_agent.create_session(user_id="jwortz")
session

Session(id='0fadada8-13c5-4b60-bd73-7f0e01dd3206', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752530795.9919639)

In [4]:
my_agent.list_sessions(user_id="jwortz")

ListSessionsResponse(sessions=[Session(id='0fadada8-13c5-4b60-bd73-7f0e01dd3206', app_name='default-app-name', user_id='jwortz', state={}, events=[], last_update_time=1752530795.9919639)])

In [ ]:
from pprint import pprint


def stream_agent(agent, prompt: str, session) -> None | Exception:
    try:
        session_id = session['id']
    # local prefers attributes vs. remote prefers dict keys
    except TypeError:
        session_id = session.id
    except Exception as e:
        return f"Session Object not valid: {e}"
        
    for event in agent.stream_query(
        user_id="jwortz",
        session_id=session_id,
        message=prompt,
    ):
        pprint(event)

In [6]:
first_prompt = f"Hello"
stream_agent(my_agent, first_prompt, session)

INFO:root:

Loading Initial State: {'state': {'artifact_keys': {'image_creatives': {}, 'video_creatives': {}}, 'target_product': 'PRS SE CE24 Electric Guitar', 'target_audience': ['Demographics: millennials who follow jam bands such as Widespread Panic and Phish.', 'Psychographics: millennials who respond positively to nostalgic messages.', 'Lifestyle or profession: frequent travelers; spending most income on concert experiences.', 'Hobbies, interests, humor: music lovers, attend lots of jam band concerts.', 'Actively researching concert and music festival tickets; musical instruments, love surreal memes (e.g.,  https://www.reddit.com/r/surrealmemes/).'], 'key_selling_points': ['Bolt-on Maple Neck - The bolt-on construction with a maple neck provides a distinct tone with enhanced brightness and a slightly more percussive attack', 'Wide Thin Profile- The Wide Thin neck carve offers a comfortable and fast playing experience, allowing for easy access to all 24 frets.', "Satin Finish - The

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'artifact_keys': {'image_creatives': {},
                                               'video_creatives': {}},
                             'campaign_guide': {'brand': 'Paul Reed Smith '
                                                         '(PRS)',
                                                'campaign_name': 'Marketing '
                                                                 'Campaign '
                                                                 'Guide: PRS '
                                                                 'SE CE24 '
                                                                 'Guitar',
                                                'campaign_objectives': ['Increase '
                                                                        'Brand '
                                       

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Hello! I'm your AI Marketing Research & Strategy Assistant. I can help you with comprehensive insights, creative ideas, and trend analysis for your marketing campaigns. We'll start by populating your campaign guide and other relevant information, then dive into current trends, conduct in-depth research, and finally, generate some amazing ad creatives for you.

Let's begin by finding some interesting trends.

-----------------------------------------------------------
Function calls:
name: transfer_to_agent, args: {'agent_name': 'trends_and_insights_agent'}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Mon, 14 Jul 2025 22:06:41 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Opt

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'request_count': 1,
                             'timer_start': 1752530798.921024}},
 'author': 'root_agent',
 'content': {'parts': [{'text': "Hello! I'm your AI Marketing Research & "
                                'Strategy Assistant. I can help you with '
                                'comprehensive insights, creative ideas, and '
                                'trend analysis for your marketing campaigns. '
                                "We'll start by populating your campaign guide "
                                'and other relevant information, then dive '
                                'into current trends, conduct in-depth '
                                'research, and finally, generate some amazing '
                                'ad creatives for you.\n'
                                '\n'
                                "Let's begin by finding some interes

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
None
-----------------------------------------------------------
Function calls:
name: get_daily_gtrends, args: {}
-----------------------------------------------------------
Raw response:
{"sdk_http_response":{"headers":{"Content-Type":"application/json; charset=UTF-8","Vary":"Referer","Content-Encoding":"gzip","Date":"Mon, 14 Jul 2025 22:06:42 GMT","Server":"scaffolding on HTTPServer2","X-XSS-Protection":"0","X-Frame-Options":"SAMEORIGIN","X-Content-Type-Options":"nosniff","Alt-Svc":"h3=\":443\"; ma=2592000,h3-29=\":443\"; ma=2592000","Transfer-Encoding":"chunked"}},"candidates":[{"content":{"parts":[{"function_call":{"args":{},"name":"get_daily_gtrends"}}],"role":"model"},"finish_reason":"STOP","avg_logprobs":-1.0006167093912761}],"create_time":"2025-07-14T22:06:42.132369Z","response_id":"cn91aJGKCKLHgLUPz8GiwAQ","model_version":"gemini-2.5-flash","usage_meta

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'function_call': {'args': {},
                                          'id': 'adk-dec63594-1844-47b6-9a57-8e00d98c7165',
                                          'name': 'get_daily_gtrends'}}],
             'role': 'model'},
 'id': '7ncSAEVD',
 'invocation_id': 'e-d4ca77f7-0ffc-4d02-adaa-34eb88eb654f',
 'long_running_tool_ids': set(),
 'timestamp': 1752530801.370821,
 'usage_metadata': {'candidates_token_count': 6,
                    'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                                   'token_count': 6}],
                    'prompt_token_count': 1746,
                    'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
                                               'token_count': 1746}],
                    'thoughts_tok

/Users/jwortz/zghost/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/jwortz/zghost/trends_and_insights_agent/common_agents/trend_assistant/tools.py:134: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return max_date.iloc[0][0].strftime("%m/%d/%Y")
INFO:root:

max_date in trends_assistant: 07/13/2025


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
INFO:google_adk.google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:

You are a helpful AI assistant, part of a multi-agent system designed for advanced web re

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'function_response': {'id': 'adk-dec63594-1844-47b6-9a57-8e00d98c7165',
                                              'name': 'get_daily_gtrends',
                                              'response': {'markdown_string_for_today_up_to_07/13/2025': '|    '
                                                                                                         '| '
                                                                                                         'term                                '
                                                                                                         '|   '
                                                                                                         'rank '
                                                                                      

INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
Here are the top 25 trending Google searches for the current week:

| | term | rank | refresh_date |
|---:|:------------------------------------|-------:|:---------------|
| 1 | chelsea - psg | 1 | 2025-07-13 |
| 2 | jannik sinner | 2 | 2025-07-13 |
| 3 | wings vs fever | 3 | 2025-07-13 |
| 4 | tems | 4 | 2025-07-13 |
| 5 | joe pavelski | 5 | 2025-07-13 |
| 6 | grand canyon lodge | 6 | 2025-07-13 |
| 7 | preston vs liverpool | 7 | 2025-07-13 |
| 8 | metlife stadium | 8 | 2025-07-13 |
| 9 | usps forever stamps | 9 | 2025-07-13 |
| 10 | mundial de clubes | 10 | 2025-07-13 |
| 11 | kentucky shooting | 11 | 2025-07-13 |
| 12 | j balvin | 12 | 2025-07-13 |
| 13 | nigerian president muhammadu buhari | 13 | 2025-07-13 |
| 14 | brody jenner | 14 | 2025-07-13 |
| 15 | mlb draft time | 15 | 2025-07-13 |
| 16 | gotterup | 16 | 2025-07-13 |
| 17 | jannik sinner brother | 17

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': 'Here are the top 25 trending Google searches '
                                'for the current week:\n'
                                '\n'
                                '| | term | rank | refresh_date |\n'
                                '|---:|:------------------------------------|-------:|:---------------|\n'
                                '| 1 | chelsea - psg | 1 | 2025-07-13 |\n'
                                '| 2 | jannik sinner | 2 | 2025-07-13 |\n'
                                '| 3 | wings vs fever | 3 | 2025-07-13 |\n'
                                '| 4 | tems | 4 | 2025-07-13 |\n'
                                '| 5 | joe pavelski | 5 | 2025-07-13 |\n'
                                '| 6 | grand canyon lodge | 6 | 2025-07-13 |\n'
                                '| 7 | preston vs li

In [ ]:
followup_prompt = "Create images from these great ideas!"
stream_agent(my_agent, followup_prompt, session)

## 4. Deploy to Vertex AI Agent Engine
To call and manage agents in production, deploy the agent to Vertex AI Agent Engine.

**Important - run `poetry build` to package the agent**

In [7]:
! poetry build --format=wheel --output=deployment

Building trends_and_insights_agent (0.1.0)
  - Building wheel
  - Built trends_and_insights_agent-0.1.0-py3-none-any.whl


#### Initialize the Vertex client, then create a `remote_agent` that is deployed to Vertex

This also takes the packaged agent code. This is required for more complex agents that have nested dependencies and require packaging

Also, before running - be sure to give Secret Manager access to the Agent Engine service account. This can simply be done by running `. setup_ae_sm_access.sh`.

The code can also be ran as follows (relative to repo root):

```bash
source trends_and_insights_agent/.env

export RE_SA="service-${GOOGLE_CLOUD_PROJECT_NUMBER}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
gcloud secrets add-iam-policy-binding "projects/$GOOGLE_CLOUD_PROJECT/secrets/$YT_SECRET_MNGR_NAME" \
  --member="serviceAccount:$RE_SA" \
  --role="roles/secretmanager.secretAccessor"
```

In [6]:
# from vertexai import agent_engines # optional cleanup

# for agent in agent_engines.list():
#     agent.delete(force=True)

In [5]:
import vertexai
from vertexai import agent_engines

GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")
BUCKET = os.getenv("BUCKET")

vertexai.init(
    project=GOOGLE_CLOUD_PROJECT,
    location="us-central1",
    staging_bucket=BUCKET,
)

remote_agent = agent_engines.create(
    agent_engine=my_agent,
    display_name="trends-and-insights",
    description="You are a helpful AI assistant, part of a multi-agent system designed for advanced web research and ad creative generation.",
    requirements=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    extra_packages=[
        "deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl",
    ],
    env_vars=env_vars,
)

Identified the following requirements: {'google-cloud-aiplatform': '1.97.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.7'}


INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.97.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.7'}


Failed to parse constraint: deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl. Exception: Expected end or semicolon (after name and no valid version specifier)
    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


    deployment/trends_and_insights_agent-0.1.0-py3-none-any.whl
              ^


The following requirements are missing: {'google-cloud-aiplatform', 'cloudpickle', 'pydantic'}


...

INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/679926387543/locations/us-central1/reasoningEngines/1065972125082320896


To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1065972125082320896')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1065972125082320896')


### Save the agent engine resource name

This can be used by the command to deploy it to Agentspace

```bash
agent_engine = vertexai.agent_engines.get('projects/679926387543/locations/us-central1/reasoningEngines/1093257605637210112')
```

## 5. Try it remotely

In [25]:
online_session = remote_agent.create_session(user_id="jwortz")
online_session

{'events': [],
 'lastUpdateTime': 1752532375.415706,
 'userId': 'jwortz',
 'id': '7581144218387611648',
 'appName': '1065972125082320896',
 'state': {}}

#### Same idea above applies to remote agents

In [23]:
from datetime import date, timedelta

# Get today's date
today = date.today()
two_days_ago = today - timedelta(days=2)

# Format the date as a string in MM/DD/YYYY format
formatted_date = two_days_ago.strftime("%m/%d/%Y")
formatted_date

'07/12/2025'

In [26]:
first_prompt = f"Hello, get the trends for today, {formatted_date}"
stream_agent(remote_agent, first_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'_state_init': True,
                             'artifact_keys': {'image_creatives': {},
                                               'video_creatives': {}},
                             'campaign_guide': {'brand': 'Paul Reed Smith '
                                                         '(PRS)',
                                                'campaign_name': 'Marketing '
                                                                 'Campaign '
                                                                 'Guide: PRS '
                                                                 'SE CE24 '
                                                                 'Guitar',
                                                'campaign_objectives': ['Increase '
                                                                        'Brand '
                                       

In [19]:
second_prompt = f"2" # select an arbitrary trend (#2 on the list for that day)
stream_agent(remote_agent, second_prompt, online_session)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {}},
 'author': 'trends_and_insights_agent',
 'content': {'parts': [{'text': "Please provide the exact 'term', 'rank', and "
                                "'refresh_date' of the trending topic you'd "
                                'like to proceed with.'}],
             'role': 'model'},
 'id': 'WBk1EkId',
 'invocation_id': 'e-c16beefb-064a-42a2-a96d-2836ea568cb6',
 'timestamp': 1752532289.151101,
 'usage_metadata': {'candidates_token_count': 28,
                    'candidates_tokens_details': [{'modality': 'TEXT',
                                                   'token_count': 28}],
                    'prompt_token_count': 1631,
                    'prompt_tokens_details': [{'modality': 'TEXT',
                                               'token_count': 1631}],
                    'thoughts_token_count': 115,
                    'total_token_count': 1774,
               

# 6. Optional Cleanup

In [ ]:
remote_agent.delete(force=True)